SCENARIO
Your client is a multinational real estate developer that builds residential and commercial properties 
around the world. They have a large portfolio of projects that are in development simultaneously. They 
currently have a 25% failure rate for their projects that is significantly higher than the industry 
benchmark of less than 10%. They would like to understand what the key leading indicators for project 
failure are when they are planning their projects. This will allow them to only invest capital into the best 
quality projects. They also want to know ongoing which projects are likely to fail so they can cut their 
losses and cease the projects. Secondly, the real estate developer would like to search local building 
policies to quickly find the relevant answers as they plan their developments. This could include zoning 
policies, environmental policies and development planning policies.

GOAL
Predict failure! Using NYC construction data.
Which ongoing projects are likely to fail?
Which projects are worth investing in? (less likely to fail)

In [3]:
import pandas as pd

df = pd.read_csv("https://datasocibmproject.s3.ap-southeast-2.amazonaws.com/structured_data/capital_project_schedules_and_budgets_1.csv")

df.head

# in case the data frame is accidentally deleted or irreversibly altered
df_copy = df.copy()

TODO:
reassigning names and datatypes; clean missing, incorrect, duplicate values, outliers

REASSIGNING NAMES AND DATATYPES

In [4]:
list(df.columns)

['Unnamed: 0',
 'project_geographic_district',
 'project_building_identifier',
 'project_school_name',
 'project_type',
 'project_description',
 'project_phase_name',
 'project_status_name',
 'project_phase_actual_start_date',
 'project_phase_planned_end_date',
 'project_phase_actual_end_date',
 'project_budget_amount',
 'final_estimate_of_actual_costs_through_end_of_phase_amount',
 'total_phase_actual_spending_amount',
 'dsf_number_s',
 'failure']

HANDLING MISSING AND INCORRECT VALUES, DUPLICATES
missing values in 
- project phase name, 
- project phase actual end date, --> also what is PNS?
- final_estimate_of_actual_costs_through_end_of_phase_amount


In [5]:
# no duplicates found
df[df.duplicated()].shape

(0, 16)